In [80]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import requests
import logging
import sys
import warnings
warnings.filterwarnings("ignore")

In [81]:
''' pip install python-dotenv'''
# load_dotenv() # will search for .env file in local folder and load variables
# Reload the variables from your .env file, overriding existing ones
load_dotenv(".env", override=True)

True

In [82]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [83]:
logger = logging.getLogger("zinc-search-call")
# Display the log message with a brown background

In [84]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized rquests. Insufficient permissions',
    404: 'the server could not find the requested resource (web page)',
    405 : 'the server has received a request using a valid HTTP method, but the target resource does not support that specific method.',
    422 : 'server understood the content type of the request body and the syntax of the request body was correct, but it was unable to process the contained instructions.',
    403 : 'Unauthorized request. Insufficient permissions. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [85]:
def get_zinc_search(url):
    ''' Search results for the ZINC Search Engine '''
    url = '{}'.format(url)
    headers = {
           'Content-type': 'application/json',
           'Authorization' : 'Basic {}'.format(os.getenv("BASIC_AUTH_ZINC_SEARCH")),
           'Connection': 'close'
    }

    # payload = {
    #     "query" : {
    #         "match_all" : {}
    #     }
    # }
    payload = {
        "search_type": "match",
        "query": {
            "term": "Paris",  # The term to search for
            "field": "_all",  # The field to search in (or a specific field name)
            # "start_time": "2021-06-02T14:28:31.894Z", # Optional: time range filtering
            # "end_time": "2021-12-02T15:28:31.894Z"
        },
        "from": 0,  # Starting point for pagination
        "max_results": 1,  # Number of results to return
        "_source": []  # List of fields to return (empty for all fields)
    }

    response = requests.post(url, json=payload, headers=headers, verify=False)
    logging.info(f"{response.status_code} - {http_status_code.get(response.status_code, '')}")
    logging.info(f"{json.dumps(response.json(), indent=2)}")

In [86]:
get_zinc_search("http://{}:4080/api/olympics/_search".format(os.getenv("api_end_point")))

2025-12-05 17:49:42,085 - root - INFO - 200 - Indicates that the request has succeeded.
2025-12-05 17:49:42,086 - root - INFO - {
  "took": 1,
  "timed_out": false,
  "_shards": {
    "total": 3,
    "successful": 3,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 2794
    },
    "max_score": 7.366430348863299,
    "hits": [
      {
        "_index": "olympics",
        "_type": "_doc",
        "_id": "2nuQSxMIsHC",
        "_score": 7.366430348863299,
        "@timestamp": "2025-12-04T21:08:45.715041024Z",
        "_source": {
          "@timestamp": "2025-12-04T21:08:45.715041024Z",
          "Athlete": "PARIS, Giuseppe",
          "City": "Paris",
          "Country": "ITA",
          "Discipline": "Artistic G.",
          "Event": "Team Competition",
          "Gender": "Men",
          "Medal": "Gold",
          "Season": "summer",
          "Sport": "Gymnastics",
          "Year": 1924
        }
      }
    ]
  }
}
